In [1]:
import json
import random
import time
import pandas as pd
import os
import argparse
import ai21
ai21.api_key = 'keys_here'
seed = 13
pairs_num = 4
model_name = 'jurassic'

In [41]:
time_stamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
random.seed(seed)

In [ ]:
import json
import random
import time
import pandas as pd
import os
import requests

df_test = pd.read_csv("../dataset/test.csv", encoding="utf-8")
df_train = pd.read_csv('../dataset/train.csv', encoding='utf-8')
test_dataset = df_test["question"]
label_dataset = df_test["label"]
train_dataset = df_train['question']
answer_dataset = df_train['answer']

test_list = []
label_list = []
for question, label in zip(test_dataset, label_dataset):
    test_list.append(question)
    label_list.append(label)


prompt = """I am a highly intelligent question answering bot. If you ask me a question that is rooted in truth, I will give you the answer. If you ask me a question that is nonsense, trickery, or has no clear answer, I will say "tricky question." first and give the reason, otherwise I will say "true question." first and give the reason."""
prompt_list = []
sample_string = ''
pos_ids = random.sample(range(int(len(train_dataset) / 2)), pairs_num)
neg_ids = [num + int(len(train_dataset) / 2) for num in pos_ids]
pos_questions = [train_dataset[id] for id in pos_ids]
neg_questions = [train_dataset[id] for id in neg_ids]
pos_answers = [answer_dataset[id] for id in pos_ids]
neg_answers = [answer_dataset[id] for id in neg_ids]


for pos_q, neg_q, pos_a, neg_a in zip(pos_questions, neg_questions, pos_answers, neg_answers):
    sample_string += f'\n\nQ: {pos_q}\nA:tricky question. {pos_a}\n\nQ: {neg_q}\nA:true question. {neg_a}'
for question in test_list:
    processed_string = f"{prompt}{sample_string}\n\nQ: {question}\nA:"
    prompt_list.append(processed_string)


print("length of prompt_list: " + str(len(prompt_list)))
print()
print('prompt and sample string: ')
print(prompt_list[0])

In [2]:
payload = {
                "numResults": 1,
                "maxTokens": 32,
                "topKReturn": 0,
                "temperature": 0.0,
                "topP": 1,
                "countPenalty": {
                    "scale": 0,
                    "applyToNumbers": False,
                    "applyToPunctuations": False,
                    "applyToStopwords": False,
                    "applyToWhitespaces": False,
                    "applyToEmojis": False
                },
                "frequencyPenalty": {
                    "scale": 0,
                    "applyToNumbers": False,
                    "applyToPunctuations": False,
                    "applyToStopwords": False,
                    "applyToWhitespaces": False,
                    "applyToEmojis": False
                },
                "presencePenalty": {
                    "scale": 0,
                    "applyToNumbers": False,
                    "applyToPunctuations": False,
                    "applyToStopwords": False,
                    "applyToWhitespaces": False,
                    "applyToEmojis": False
                },
                "stopSequences": ["\n"]
            }

In [44]:
result_list = []
response_list = []
count = 0

In [ ]:
for prompt in prompt_list[count:]:
    response = ai21.Completion.execute(model="j1-jumbo", prompt=prompt, **payload)
    response_list.append(response)
    count += 1
    time.sleep(1)
    if count % 200 == 0:
        print(count)

In [ ]:
print(count)
print(len(response_list))
print([response_list[count - 1]['completions'][0]['data']['text']])

In [48]:
new_response_list = []
for response in response_list:
    new_response_list.append(response.values)

In [49]:
with open(f"../output/exp-3/incontext/exp-3_jurrasic_scale-{pairs_num}_{model_name}_{time_stamp}_{str(seed)}.json", "a") as f:
    for i in range(len(new_response_list)):
        new_response_list[i]["question"] = test_list[i]
        new_response_list[i]["label"] = label_list[i]
    json.dump(new_response_list, f, indent=4)

In [ ]:
with open(f"../output/exp-3/incontext/exp-3_jurrasic_scale-{pairs_num}_{model_name}_{time_stamp}_{str(seed)}.json", "r") as f:
    output_dict = {'question': [], 'answer': [], 'label': []}
    data = json.load(f)
    for response in data:
        label = response['label']
        question = response['question']
        answer = response['completions'][0]['data']['text']
        output_dict['label'].append(label)
        output_dict['question'].append(question)
        output_dict['answer'].append(answer.replace("\n", ''))
df = pd.DataFrame(output_dict)
df.to_csv(f"../output/exp-3/incontext/exp-3_jurrasic_scale-{pairs_num}_{model_name}_{time_stamp}_{str(seed)}.csv")
print('result saved in ' + f"../output/exp-3/incontext/exp-3_jurrasic_scale-{pairs_num}_{model_name}_{time_stamp}_{str(seed)}.csv")